# Notebook to implement and test a simple Runge Kutta Solver

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg


rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()

In [ ]:
#Testproblem: Harmonic Oscilator

def f_A(t,u):
    A = np.array([[0,-1],[1,0]])
    c = np.array([1,1])
    #print(u)
    return A@(u-c)

u0 =np.array([1.,0.])


In [ ]:
# Reference implementation from Nodepy
from nodepy import ivp, rk

rk_ssp104 = rk.loadRKM('SSP104')


testp = ivp.IVP(f=f_A,u0=u0,T=100)

t,u =rk_ssp104(testp,dt=0.1)

plt.plot(t,u)

In [ ]:
def f(u):
    return f_A(0,u)

t, u = RRK_pos(ssp104,dt=0.1,f=f,w0=u0,t_final=100.,relaxation=False)

plt.plot(t,u.T)

In [ ]:
def RK(rkm, dt, f, w0=[1.,0], t_final=1.):
    """    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time        
    """
    
    #setup Variables for Soulution storage
    p = len(w0) #number of dimentions
    
    uu = np.zeros([p,int(t_final/dt)+2])
    uu[:,0] = w0.copy()
    tt = np.zeros([int(t_final/dt)+2])
    
    
    #Setup Runge Kutta 
    c = rkm.c
    A = rkm.A #has to be lower left triangle
    b =rkm.b
    s = len(c) #number of Stages
    K = np.zeros([p,s])
    
    u = np.array(w0)
    t = 0.
    n = 0
        
    #print('set up starting to solve')
    
    #Solve ODE
    while t<t_final:
        for i in range(s):
            #compute Stages
            
                
            #K[:,i] = f(t+c[i]*dt,u+dt*K@A[i,:]) 
            #the 0s in A should make shure that no data from an older Step is used
            
            #Maybe better Approach, because A[i,j] = 0 in many places
            u_prime = u.copy()
            for m in range(i):  #Error in range(i-1)
                u_prime += dt*A[i,m]*K[:,m]
            
            K[:,i] = f(t+c[i]*dt,u_prime)
            
            #print('intermediatestep computed')
        

            
        #update
        u += dt*K@b
        n += 1
        t += dt
        
        uu[:,n] = u.copy()
        tt[n] = t
        #print('updated')

        
    return (tt[0:n],uu[:,0:n])
        
        



In [ ]:
t, u = RK(ssp104,dt=0.1,f=f_A,w0=u0,t_final=100.)

plt.plot(t,u.T)

In [ ]:
#Testproblem from nodepy

myivp = ivp.detest('A1')
rk_rk4 = rk.loadRKM('RK44')
t_ref, u_ref = rk_rk4(myivp)


t, u = RK(rk4,dt=float(myivp.dt0),f=myivp.rhs,w0=np.array([float(myivp.u0)]),t_final=myivp.T)


plt.plot(t_ref,u_ref)
plt.plot(t,u)
